In [ ]:
! pip install spacy
! pip install pandas

Prepare spaCy-format training data for two separate models

In [ ]:
import random
import pandas as pd

# Load the dataset (CSV file must have columns like: description, type, category)
df = pd.read_csv("expenses_income_dataset.csv")


In [ ]:
# Define possible labels for the "type" classification (binary: Expense or Income)
type_labels = ["Expense", "Income"]
# Define possible categories for "Expense" type
expense_cats = ["Food", "Transport", "Rent", "Entertainment", "Shopping", "Bills"]
# Define possible categories for "Income" type
income_cats = ["Salary", "Freelance", "Sale", "Investment", "Refund"]

# Combine expense and income categories into one list
all_cats = expense_cats + income_cats


In [ ]:
# Prepare training lists
train_data_type = []   # For classifying whether text is "Expense" or "Income"
train_data_cat = []    # For classifying into detailed categories

In [ ]:
# Train two spaCy text categorizer models (type and category)